In [ ]:
import os
import glob2
import json
import ast
from pprint import pprint
import folium
from folium import plugins

In [ ]:
data_path = '/srv/data/db/persons/'
glob_search_path = os.path.join(data_path, "**/*measure*.json")
json_paths = glob2.glob(glob_search_path)
print(len(json_paths))

In [ ]:
dataset_path = '/srv/data/'

In [ ]:
def get_json_paths(dataset_path):
        glob_search_path = os.path.join(dataset_path, "db/**/*.json")
        json_paths = glob2.glob(glob_search_path)
        json_paths_personal = [
            json_path for json_path in json_paths
            if "measures" not in json_path
        ]
        json_paths_measures = [
            json_path for json_path in json_paths if "measures" in json_path
        ]
        return json_paths_personal, json_paths_measures


In [ ]:
    def _find_qrcodes(json_paths_personal, json_paths_measures):
        """
        Finds all QR-codes.

        Each individual is represented via a unique QR-codes. This method extracts the set of QR-codes.
        """
        # Go through all the measures and extract their QR-codes.
        qrcodes = []
        for json_path_measure in json_paths_measures:
            with open(json_path_measure) as json_path_measure_file:
                json_data_measure = json.load(json_path_measure_file)
                qrcode = _extract_qrcode(json_paths_personal, json_data_measure)
                if qrcode is None:
                    continue
                qrcodes.append(qrcode)

        # Provide a sorted set.
        qrcodes = sorted(list(set(qrcodes)))
        return qrcodes

In [ ]:
    def _extract_qrcode(json_paths_personal, json_data_measure):
        """
        Extracts a QR-code from a JSON.
        """
        person_id = json_data_measure["personId"]["value"]
        #log.debug("Processing person id %s" % str(person_id))
        json_path_personal = [
            json_path for json_path in json_paths_personal
            if person_id in json_path
        ]
        #log.debug("Json path personal for person %s = %s" %
        #          (person_id, str(json_path_personal)))
        #assert len(json_path_personal) == 1
        if len(json_path_personal) == 0:
            #log.warning("Cannot create json path personal for person %s " %
            #            person_id)
            return None
        if len(json_path_personal) > 1:
            print("More than 1 json path personal for person %s" % person_id)
            #log.warning("More than 1 json path personal for person %s" %
            #            person_id)

        json_path_personal = json_path_personal[0]
        try:
            json_data_personal = json.load(open(json_path_personal))
        except Exception:
            #log.exception("Unable to json load file %s" % json_path_personal)
            return None

        qrcode = json_data_personal["qrcode"]["value"]

        return qrcode

In [ ]:
    def _extract_targets(output_targets, json_data_measure):
        """
        Extracts a list of targets from JSON.
        """

        targets = []
        for output_target in output_targets:
            #print(output_target)
            value = json_data_measure[output_target]["value"]
            targets.append(value)
        return targets

In [ ]:

def create_qrcodes_dictionary(json_paths_personal, json_paths_measures):
    """
    Creates a QR-Code-dictionary.

    This basically sorts all PCDs and JPGs.
    With respect to the targets and the QR-Codes.
    This is used heavily during data generation.
    Takes into account timestamps in order to connect data and measures.
    """
    output_targets = ["timestamp","location"]#,"latitude","longitude"]

    qrcodes_dictionary = {}
    print("inside create_qrcodes_dictionary ")
    print("Total number of json paths measures %d" %
             len(json_paths_measures))

    # Go thorugh all measures.
    for json_path_measure in json_paths_measures:
        #print("Processing json path measure file %s" %
        #         str(json_path_measure))

        # Load the data and get type.
        json_data_measure = json.load(open(json_path_measure))
        measure_type = json_data_measure["type"]["value"]
        # Ensure manual data. If it is not a manual measurement, skip.
        #if measure_type != "manual":
        #    print("Ignoring measure file %s as type != manual" %
        #                str(json_path_measure))
        #    continue

        # Extract the QR-code.
        qrcode = _extract_qrcode(json_paths_personal, json_data_measure)
        if qrcode is None:
            continue
        #print("Extracted qr code %s" % str(qrcode))

        # Create an array in the dictionary if necessary.
        # Extract the targets from the JSON-data.
        targets = _extract_targets(output_targets, json_data_measure)
        #print("Extracted targets from file %s %s" %
        #          (json_path_measure, str(targets)))
        '''
        # Extract the timestamp from the JSON-data.
        timestamp = etl_utils.extract_timestamp_from_path(
            json_path_measure)

        # Filter paths for qrcodes and measurements.
        # Find all JPGs and PCDs for a given QR-code and make sure that the timestamps are related.
        
        jpg_paths = [
            jpg_path for jpg_path in self.jpg_paths
            if etl_utils.is_matching_measurement(jpg_path, qrcode,
                                                 timestamp)
        ]
        print("Extracted JPG paths for file %s : %d" %
                  (str(json_path_measure), len(jpg_paths)))

        pcd_paths = [
            pcd_path for pcd_path in self.pcd_paths
            if etl_utils.is_matching_measurement(pcd_path, qrcode,
                                                 timestamp)
        ]
        print("Extracted pcd_path for file %s : %d" %
                  (str(json_path_measure), len(pcd_paths)))

        if len(pcd_paths) == 0:
            print("Ignoring qr code %s as pcd paths are empty" %
                        str(qrcode))
            continue
        '''

        #print("Added qr code %s into dictionary with targets %s" %
        #         (qrcode, str(targets)))
        if qrcode not in qrcodes_dictionary.keys():
            qrcodes_dictionary[qrcode] = []

        qrcodes_dictionary[qrcode].append((targets))#, jpg_paths, pcd_paths, timestamp))

    return qrcodes_dictionary


In [ ]:
json_paths_personal, json_paths_measures = get_json_paths(dataset_path)

In [ ]:
qdict = create_qrcodes_dictionary(json_paths_personal, json_paths_measures)

In [ ]:
latlong = []
for qr in qdict:
    
    if qr is None:
        continue
    
    if qdict[qr][0][1] is None:
        continue
    
    if qdict[qr][0][1] is None:
        continue
    
    timestamp = qdict[qr][0][0]
    lat = qdict[qr][0][1]['latitude']['value']
    long = qdict[qr][0][1]['longitude']['value']
    latlong.append([lat,long])
    #print(qr+","+str(timestamp)+","+str(lat)+","+str(long))

#pprint(qdict["MH_WHH_0047"][0][1]['latitude']['value'])
#pprint(ast.literal_eval(qdict["MH_WHH_0047"][0][1]))
#j = json.loads(str(ast.literal_eval(str(qdict["MH_WHH_0047"][0][1]))))
#print(qdict.keys())

In [ ]:
mapit = None
#latlon = [ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
mapit = folium.Map( location=[22.4, 75.2], zoom_start=6 )
#for coord in latlong:
#    folium.Marker( location=[ coord[0], coord[1] ]).add_to( mapit )

mapit.add_child(plugins.HeatMap(latlong, radius=15))
mapit